In [ ]:
# working on reading the BME280.  The way it achieves its lack of noise is with oversampling and filters
# We turn all these off so we can process the values with our own noise handling
# And set it to sample as fast as possible in normal mode, which is a rate of about 10ms

In [1]:
%serialconnect


Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.


In [2]:
%sendtofile --source bme280.py

Sent 87 lines (3502 bytes) to bme280.py.


[119] 0x77


In [8]:
# use this to check
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

# SDI is SDA(pin4), SCK is SCL(pin5) 
print(i2c.scan(), hex(119))

from bme280 import bme280gen
a = bme280gen(i2c)

[119] 0x77


['__init__.mpy', 'core.mpy', 'synchro.mpy']


In [21]:
for i in range(10):
    print(next(a))


(19.23, 101001.8, 30.46484)
(19.23, 101001.8, 30.46484)
(19.23, 101001.8, 30.46484)
(19.23, 101001.8, 30.46484)
(19.23, 101001.8, 30.46484)
(19.24, 101006.1, 30.5332)
(19.24, 101006.1, 30.5332)
(19.24, 101006.1, 30.5332)
(19.24, 101006.1, 30.5332)
(19.24, 101006.1, 30.5332)


In [182]:
# the following for working out the bme280.py file
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))


def bme280_checkchipid():
    k = i2c.readfrom_mem(0x77, 0xD0, 1)
    return (k == b'\x60')

print(i2c.scan())
print(bme280_checkchipid())

[119]
True


In [168]:
# ctlr_hum  0xF2: none none none none none osrs_h2 osrs_h1 osrs_h0
i2c.writeto_mem(0x77, 0xF2, b'\x01')  # switch on humidity (over)sampling=1

# ctrl_meas 0xF4: osrs_t2 osrs_t1 osrs_t0 osrs_p2 osrs_p1 osrs_p0 mode1 mode0
i2c.writeto_mem(0x77, 0xF4, b'\x27')  # switch on temp and pressure (over)sampling=1, mode=normal(continuous readings)

# config    0xF5: t_sb2 t_sb1 t_sb0 filter2 filter1 filter0 none spi3w_en0
i2c.writeto_mem(0x77, 0xF5, b'\x00')  # 0.5ms standby, no filtering, no-SPI

# status    0xF3: none none none none measuring0 none none im_update0
# this doesn't seem to get 1 values ever


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
OSError: [Errno 19] ENODEV


In [167]:
# pack the 19 calibration numbers down efficiently into one structure
import array, ustruct

dT1, dT2, dT3 = 0, 1, 2 #const(0), const(1), const(2)
dP1, dP2, dP3, dP4, dP5, dP6, dP7, dP8, dP9 = 3, 4, 5, 6, 7, 8, 9, 10, 11 #const(0), const(1), const(2)
dH1, dH2, dH3, dH4, dH5, dH6 = 13, 14, 15, 16, 17, 18

def GetBME280calibrations():
    dc = array.array("i", range(19))
    dc[dT1], dc[dT2], dc[dT3] = ustruct.unpack("<Hhh", i2c.readfrom_mem(0x77, 0x88, 6))
    dc[dP1], dc[dP2], dc[dP3], dc[dP4], dc[dP5], dc[dP6], dc[dP7], dc[dP8], dc[dP9] = \
                            ustruct.unpack("<Hhhhhhhhh", i2c.readfrom_mem(0x77, 0x8E, 18))
    dc[dH1] = i2c.readfrom_mem(0x77, 0xA1, 1)[0]
    dig_e1_e7 = i2c.readfrom_mem(0x77, 0xE1, 7)
    dc[dH2], dc[dH3] = ustruct.unpack("<hB", dig_e1_e7)
    e4_sign = ustruct.unpack_from("<b", dig_e1_e7, 3)[0]
    dc[dH4] = (e4_sign << 4) | (dig_e1_e7[4] & 0xF)
    e6_sign = ustruct.unpack_from("<b", dig_e1_e7, 5)[0]
    dc[dH5] = (e6_sign << 4) | (dig_e1_e7[4] >> 4)
    dc[dH6] = ustruct.unpack_from("<b", dig_e1_e7, 6)[0]
    return dc

dc = GetBME280calibrations()
print(dc)

Traceback (most recent call last):
  File "<stdin>", line 23, in <module>
  File "<stdin>", line 10, in GetBME280calibrations
OSError: [Errno 19] ENODEV


In [155]:
readout = bytearray(8)
tph = array.array("i", range(3))

def readBME280():
    # burst readout from 0xF7 to 0xFE, recommended by datasheet
    i2c.readfrom_mem_into(0x77, 0xF7, readout)
    raw_press = ((readout[0] << 16) | (readout[1] << 8) | readout[2]) >> 4
    raw_temp = ((readout[3] << 16) | (readout[4] << 8) | readout[5]) >> 4
    raw_hum = (readout[6] << 8) | readout[7]

    # temperature
    var1 = ((raw_temp >> 3) - (dc[dT1] << 1)) * (dc[dT2] >> 11)
    var2 = (((((raw_temp >> 4) - dc[dT1]) * ((raw_temp >> 4) - dc[dT1])) >> 12) * dc[dT3]) >> 14
    t_fine = var1 + var2
    tph[0] = (t_fine * 5 + 128) >> 8
    
    # pressure
    var1 = t_fine - 128000
    var2 = var1 * var1 * dc[dP6]
    var2 = var2 + ((var1 * dc[dP5]) << 17)
    var2 = var2 + (dc[dP4] << 35)
    var1 = (((var1 * var1 * dc[dP3]) >> 8) + ((var1 * dc[dP2]) << 12))
    var1 = (((1 << 47) + var1) * dc[dP1]) >> 33
    if var1 == 0:
        tph[1] = 0
    else:
        p = 1048576 - raw_press
        p = (((p << 31) - var2) * 3125) // var1
        var1 = (dc[dP9] * (p >> 13) * (p >> 13)) >> 25
        var2 = (dc[dP8] * p) >> 19
        tph[1] = ((p + var1 + var2) >> 8) + (dc[dP7] << 4)
    
    # humidity
    h = t_fine - 76800
    h = (((((raw_hum << 14) - (dc[dH4] << 20) - (dc[dH5] * h)) + 16384) >> 15) * (((((((h * dc[dH6]) >> 10) * (((h * dc[dH3]) >> 11) + 32768)) >> 10) + 2097152) * dc[dH2] + 8192) >> 14))
    h = h - (((((h >> 15) * (h >> 15)) >> 7) * dc[dH1]) >> 4)
    h = 0 if h < 0 else h
    h = 419430400 if h > 419430400 else h
    tph[2] = h >> 12




In [159]:
from connecthotspot import ipnumber


I (166869) network: event 13
Creating access point ESP_0e3acd
Device has ipnumber 192.168.4.1


In [160]:
for i in range(20):
    readBME280()
    print(tph[0]/100, tph[1]/256, tph[2]/1024)
    time.sleep_ms(500)



18.64 101899.5 39.02442
18.64 101891.4 39.03613
18.64 101896.8 39.04688
18.64 101897.5 39.04785
18.65 101892.9 39.03711
18.64 101892.1 39.04785
I (179439) network: event 15
18.64 101896.8 39.03613
18.65 101892.9 39.03711
18.64 101892.1 39.03711
18.64 101888.7 39.02442
18.64 101894.8 39.05957
18.64 101894.8 39.04785
18.65 101898.3 39.02539
18.64 101894.8 39.04785
18.64 101894.1 39.02442
18.65 101892.9 39.05957
18.66 101897.2 39.04883
18.66 101889.1 39.06055
18.67 101896.8 39.16211
18.67 101896.8 39.36523


In [34]:
import machine

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
bme = BME280(i2c=i2c)

print(bme.values)


('20.3C', '711.12hPa', '76.57%')


In [35]:
import time
for i in range(10):
    print(bme.values)
    time.sleep(1)
    

('22.93C', '1017.35hPa', '40.22%')
('22.89C', '1017.32hPa', '40.26%')
('22.85C', '1017.37hPa', '40.25%')
('23.15C', '1017.29hPa', '43.45%')
('23.0C', '1017.34hPa', '42.25%')
.('23.24C', '1017.31hPa', '54.34%')
('22.91C', '1017.35hPa', '59.50%')
('22.84C', '1017.40hPa', '54.74%')
('22.8C', '1017.45hPa', '49.93%')
('22.77C', '1017.39hPa', '47.09%')


In [113]:
%%writefile bme280.py

thing

Writing bme280.py



In [ ]:
# code taken from other main

# hook = [ go=1/exit=0, time, t, p, h ]
async def abme280(i2c, hook):
    log("Entering abme280")
    try:
        if not bme280init(i2c):
            raise Exception("bme280 not found")
        dc = GetBME280calibrations(i2c)
        readout = bytearray(8)
        tph = array.array("i", range(3))
        while hook[0]:
            hook[1] = time.ticks_ms()
            readBME280(tph, dc, readout, i2c)
            hook[2] = tph[0]; hook[3] = tph[1]; hook[4] = tph[2]; 
            await uasyncio.sleep_ms(12)
    except Exception as e:
        elog(e)
    hook[0] = -1
    log("Leaving abme280")

    
def conv4inttoHex(hexbuff, i1, i2, i3, i4):
    ustruct.pack_into(">IIII", hexbuff, 16, i1, i2, i3, i4)
    for i in range(16):
        b = hexbuff[i+16]
        hexbuff[i*2] = (b>>4) + (48 if b < 160 else 55)  #ord('0')=48; ord('A')=65
        b &= 15
        hexbuff[i*2+1] = b + (48 if b < 10 else 55)

    
    

In [ ]:
# code not yet finished

class SensorBME280:
    def __init__(self):
        self.gocount = 1   # set to 0 to cause to end
        self.t = time.ticks_ms()
        self.tph = array.array("i", range(3))
        self.fout = None
    
    async def task(i2c):
        log("Entering abme280")
        try:
            if not bme280init(i2c):
                raise Exception("bme280 not found")
            dc = GetBME280calibrations(i2c)
            readout = bytearray(8)
            tph = array.array("i", range(3))
            while hook[0]:
                hook[1] = time.ticks_ms()
                readBME280(tph, dc, readout, i2c)
                hook[2] = tph[0]; hook[3] = tph[1]; hook[4] = tph[2]; 
                await uasyncio.sleep_ms(12)
        except Exception as e:
            elog(e)
        self.gocount = 0
        log("Leaving abme280")
